# Einführung Neuronale Netze (Deep Learning)

## Wiederholung

Blick auf die Ergebnissse, die wir mit Schätzung und einem einfachen Machine Learning Modell erzielt haben

In [ ]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP
from micrograd.graph import draw_dot

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/eis.csv')
df.head(10)

In [ ]:
print("Durchschnittlicher Fehler")
print(f'Schätzung {np.mean(df["Fehler"].to_list())}')
print(f'Machine Learning {np.mean(df["Fehler_ML"].to_list())}')

In [ ]:
ax=df.plot.scatter(x='Temperatur', y=['Eis'], c='blue')
df.plot.scatter(x='Temperatur', y=['Eis_Vorhersage'], c='red',ax=ax)
df.plot.scatter(x='Temperatur', y=['Eis_Vorhersage_ML'], c='purple',ax=ax)
plt.show()

#### Aufgabe 1

##### 3 Punkte

- F: Beschreiben Sie das Modell das wir trainiert haben in eigenen Worten (es geht auch in genau einem Wort):
- A:
- F: Wieviele Parameter hat unser erstes Modell und wie haben wir diese genannt (sie können Sie auch beschreiben)?
- A:
- F: Beschreiben Sie, was wir unter dem Fehler verstehen in eigenen Worten
- A:

## Natürliche und künstliche Neuronen

<img src="img/04DL00.png" alt="drawing" style="width:800px;"/>    

In [ ]:
def berechne_fehler(ist, vorhersage):
    fehler_liste = []
    for ist_wert, vorhersage_wert in zip(ist, vorhersage):
        fehler = (ist_wert - vorhersage_wert) * (ist_wert - vorhersage_wert)
        fehler_liste.append(fehler)
    return fehler_liste

In [ ]:
def mache_vorhersage(eingangssignale, nn):
    ausgangssignale = list(map(nn, eingangssignale))
    return ausgangssignale

In [ ]:
def suche(nn, uvar, ist, lernrate, schritte):
    ist = np.atleast_2d(np.array(ist, dtype=np.float32)).T
    eingangssignale = np.atleast_2d(np.array(uvar, dtype=np.float32)).T
    for i in range(schritte): 
        vorhersage = mache_vorhersage(eingangssignale, nn)
        fehler = np.sum(berechne_fehler(ist, vorhersage)) / len(ist)

        nn.zero_grad()
        fehler.backward()

        for p in nn.parameters():
            p.data -= lernrate * p.grad

        if i % 100 == 0:
            print(f"step {i} loss {fehler.data}")    

## Ein einsames Neuron

In [ ]:
n = Neuron(1, nonlin=None)
print(n)
print("number of parameters", len(n.parameters()))

In [ ]:
temp = 20

In [ ]:
eis = n([temp])
w_initial = n.parameters()[0].data
b_initial = n.parameters()[1].data
print(f"eis = {temp} * {w_initial} + {b_initial} = {eis.data}")

In [ ]:
draw_dot(eis)

#### Aufgabe 2

##### 2 Punkte

Führen Sie alle vier Zellen oberhalb dieser Aufgabe wiederholt hintereinander aus.

- F: Was verändert sich dabei alles?
- A:
- F: Was ist der zusammenhang zwischen dem Schaubild oben und der Rechnung darüber?
- A:

## Ein Neuron lernt

In [ ]:
n = Neuron(1, nonlin=None)
suche(n, df["Temperatur"].to_list(), df["Eis"].to_list(), 0.002, 1)
eis = n([temp])
draw_dot(eis)

In [ ]:
suche(n, df["Temperatur"].to_list(), df["Eis"].to_list(), 0.002, 1)
eis = n([temp])
draw_dot(eis)

In [ ]:
suche(n, df["Temperatur"].to_list(), df["Eis"].to_list(), 0.002, 1)
eis = n([temp])
draw_dot(eis)

#### Aufgabe 3

##### 2 Punkte

Vergleichen Sie die drei Lernschritte oben. 

- F: Wie verändern sich die Daten des Parameters links unten und in der Mitte oben im Vergleich zum jeweiligen Gradient?
- A:
- F: Beschreiben Sie die Wirkung des Gradienten in eigenen Wortenm
- A:

### Wir gehen wieder viele Schritte

In [ ]:
lernrate = 0.002
schritte = 1000

In [ ]:
suche(n, df["Temperatur"].to_list(), df["Eis"].to_list(), lernrate, schritte)
w_trainiert = n.parameters()[0].data
b_trainiert = n.parameters()[1].data
eis = n([temp])
print(f"eis = {temp} * {w_trainiert} + {b_trainiert} = {eis.data}")

In [ ]:
draw_dot(eis)

In [ ]:
eingangssignale = np.atleast_2d(df["Temperatur"].to_numpy(dtype=np.float32)).T
df["Eis_Vorhersage_N"] = [v.data for v in mache_vorhersage(eingangssignale, n)]
df["Fehler_N"] = berechne_fehler(df["Eis"].to_list(), df["Eis_Vorhersage_N"].to_list())
df.head(10)

In [ ]:
print(np.mean(df["Fehler_N"].to_list()))
if "Fehler_ML" in df:
    print(np.mean(df["Fehler_ML"].to_list()))
if "Fehler" in df:
    print(np.mean(df["Fehler"].to_list()))

In [ ]:
eingangssignale = np.atleast_2d(np.asarray([0, 40], dtype=np.float32)).T
vorhersagen = [v.data for v in mache_vorhersage(eingangssignale, n)]
plt.scatter(df["Temperatur"], df["Eis"])
plt.plot([0, 40], [vorhersagen[0], vorhersagen[1]], color='purple') 
plt.xlabel("Temperatur")
plt.ylabel("Eis")
plt.show()

## Ein Netz von Neuronen

In [ ]:
nn = MLP(1, [4,1], 'relu') # 2-layer neural network
print(nn)
print("number of parameters", len(nn.parameters()))

In [ ]:
eis = nn([temp])
print(f"eis = {eis.data}")

In [ ]:
draw_dot(eis)

In [ ]:
lernrate = 0.0001 #0.002
schritte = 10000 #2000
suche(nn, df["Temperatur"].to_list(), df["Eis"].to_list(), lernrate, schritte)

#### Aufgabe 4

##### 1 Punkte

Führen Sie das experiment oben mehrfach durch. Dazu müssen sie immer wieder in der Zelle unterhalb der Überschrift "Ein Netz von Neuronen" beginnen.

- F: Was ist ihr bestes Ergebniss (loss)?
- A:
- F: Warum unterscheiden sich die Ergebnisse bei jedem Versuch?
- A:
- F: Warum brauchen wir viel mehr Schritte als bei unserem ersten einfachen Modell?
- A:

## Abschluss

In [ ]:
eingangssignale = np.atleast_2d(df["Temperatur"].to_numpy(dtype=np.float32)).T
df["Eis_Vorhersage_NN"] = [v.data for v in mache_vorhersage(eingangssignale, nn)]
df["Fehler_NN"] = berechne_fehler(df["Eis"].to_list(), df["Eis_Vorhersage_NN"].to_list())
df.head(10)

In [ ]:
xRange = np.atleast_2d(np.arange(0, 40, 1).tolist()).T
inputs = [list(map(Value, xrow)) for xrow in xRange]
scores = [y.data for y in list(map(nn, inputs))]
plt.scatter(df["Temperatur"], df["Eis"])
plt.plot(xRange, scores, color='purple') 
plt.xlabel("Temperatur")
plt.ylabel("Eis")
plt.show()
#print(scores)

#### Aufgabe 5

##### 1 Punkt

Schauen Sie auf die Grafik oben.

- F: Wie unterscheidet sich unser neues Modell aus Neuronen graphisch von unserem ersten Machine Learning Modell? 
- A:
